In [1]:
import math
import pandas

|Juggments|id|
|--------|--|
|4|pp1|
|3|pp2|
|1|pp3|
|1|pp4|
|0|pp5|

$$Reranking\ result=[pp3, pp2, pp1, pp5, pp4]$$
$$Reranked=[1, 3, 4, 0,1]$$
$$k=5$$

In [10]:
reranked = [1, 3, 4 ,0, 1]
max_rank = 4

$$Precision@k=\frac{number\ of\ relevant}{total\ number}$$

In [11]:
precision = sum(1 for rank in reranked if rank > 0) / len(reranked)
print('Precision={:.2f}'.format(precision))

Precision=0.80


$$NDCG@k=\frac{DCG@k}{Ideal\ DCG@k}$$
$$DCG@k=\sum_{i=1}^k \frac{2^{rel_i}-1}{log_2(i+1)}$$

In [8]:
def d_g(rank_list):
    return [(pow(2, rel) - 1)/(math.log2(i + 2)) for i, rel in enumerate(rank_list)]


DG = d_g(reranked)
ideal = sorted(reranked, reverse=True)
IDG = d_g(ideal)

DCG = sum(DG)
IDCG = sum(IDG)
NDCG = DCG / IDCG 

df_dcg = pandas.DataFrame({
   'rel_i': reranked,
   'DG': DG
})
df_idcg = pandas.DataFrame({
   'rel_i': ideal,
   'IDG': IDG
})

print("===== Reranked ====")
print(df_dcg)
print("===== Ideal ====")
print(df_idcg)
print("===== Result ====")
print('DCG={:.2f}, Ideal DCG={:.2f}, NDCG={:.2f}'.format(DCG, IDCG, NDCG))

===== Reranked ====
   rel_i        DG
0      1  1.000000
1      3  4.416508
2      4  7.500000
3      0  0.000000
4      1  0.386853
===== Ideal ====
   rel_i        IDG
0      4  15.000000
1      3   4.416508
2      1   0.500000
3      1   0.430677
4      0   0.000000
===== Result ====
DCG=13.30, Ideal DCG=20.35, NDCG=0.65


$$ERR=\sum_{i=1}^k \frac{1}{i} P(user\ stops\ at\ position\ i)$$
\begin{align}
ERR=\sum_{i=1}^k \frac{1}{i} \prod_{j=1}^{i-1}(1-R_j)R_i &&
R = \frac{2^{rel} - 1}{2^{max\_rel}}
\end{align}

In [12]:
ERR = 0.0
err_list = []
prob_step_down = 1.0
prob_list = []
    
for r, rel in enumerate(reranked):
    r += 1
    R = (pow(2, rel) - 1) / pow(2, max_rank)
    err_step = prob_step_down * R / r
    ERR += err_step
    err_list.append(err_step)

    prob_step_down *= (1 - R)
    prob_list.append(prob_step_down)
    
df_ERR = pandas.DataFrame({
   'probability': prob_list,
   'ERR': err_list
})

print("===== Iterations ====")
print(df_ERR)
print("===== Result ====")
print('ERR={:.2f}'.format(ERR))

===== Iterations ====
   probability       ERR
0     0.937500  0.062500
1     0.527344  0.205078
2     0.032959  0.164795
3     0.032959  0.000000
4     0.030899  0.000412
===== Result ====
ERR=0.43
